# Wanted Crawling Program

In [18]:
# 함수코드

import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementClickInterceptedException
import re
import time


# 함수 생성
def wanted_crawling_program(start_post, end_post):
    # 크롬옵션 설정
    chrome_options = Options()
    # 꺼짐 방지 (gqu가속 x)
    chrome_options.add_argument('--disable-gpu') 
    # 창 크기에 따라 XPATH가 달라져 크기 고정
    chrome_options.add_argument("--start-maximized")
    # 알림표시 제거
    chrome_options.add_argument("--disable-notifications")
    # 옵션 적용
    driver = webdriver.Chrome(options=chrome_options)
    
    url = "https://www.wanted.co.kr/wdlist/518?country=kr&job_sort=job.recommend_order&years=-1&locations=all"
    # chrome url 넣어서 실행
    driver.get(url)

    # 최종 Data가 모일 곳
    wanted_res = pd.DataFrame(
        columns = ['기업명', '주소', '직무', '자격요건', '직급', '이용하는기술스택/우대사항', '해당 페이지 URL']
        )

    # 직무 선택 클릭
    search_job = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located(
        (
            By.XPATH, '//*[@id="__next"]/div[3]/article/div/div[2]/button'
            )
            )
            )
    search_job.click()
    


    # 이 함수는 공고를 4회 클릭시 로그인창이 발생하는 사이트의 특성 때문에
    # 89 번째 줄의 try에서 3개의 공고를 클릭하여 크롤링한 후 
    # finally에서 driver를 종료시킨 다음 드라이브를 켜 반복을 이어갑니다.
    # 직무선택 변수 button의 for반복문은 이 곳에 넣을 경우 계속 초기화 되기에 
    # '함수의 실행코드' 에 포함시켰습니다.
    driver.find_element(
        By.XPATH, f'//*[@id="__next"]/div[3]/article/div/div[2]/section/div[1]/div/button[{button}]'
        ).click()
    driver.find_element(
        By.XPATH, '//*[@id="__next"]/div[3]/article/div/div[2]/section/div[2]/button/span[2]'
        ).click()
    
    
    # aws 검색하기
    search = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located(
            (
                By.XPATH, '//*[@id="__next"]/div[3]/div[1]/div[1]/div/div/div[2]/button'
                )
                )
                )
    search.click()
    # aws 입력        
    time.sleep(0.5)
    driver.find_element(
        By.XPATH, '//*[@id="__next"]/div[3]/div[1]/div/div/div/div[2]/section/div/div[1]/input'
        ).send_keys('aws')
    # 엔터        
    time.sleep(0.5)
    driver.find_element(
        By.XPATH, '//*[@id="__next"]/div[3]/div[1]/div/div/div/div[2]/section/div/div[1]/input'
        ).send_keys(Keys.ENTER)
    # 확인       
    time.sleep(0.7)
    driver.find_element(
        By.XPATH, '//*[@id="__next"]/div[3]/div[1]/div/div/div/div[2]/section/footer/div/button[2]/span[2]'
        ).click()
            
    time.sleep(1)
    

    start_time_scroll = time.time()
    
    # 크롤링 코드블럭
    try:
        for post_job in range(start_post, end_post):
            if time.time() - start_time_scroll > 60:
                break 
            #스크롤하며 공고 찾기
            while True:
                try:
                    # 공고 찾기
                    post = driver.find_element(
                         By.XPATH, f'//*[@id="__next"]/div[3]/div[2]/ul/li[{post_job}]/div/a/div[1]'
                         )
                    # 있으면 클릭하고 break
                    time.sleep(0.1)
                    try:
                        if post.is_displayed():
                            post.click()
                            break
                    except ElementClickInterceptedException:
                        driver.execute_script("window.scrollBy(0, 100);")
                        if post.is_displayed():
                            post.click()
                            break
                except ElementClickInterceptedException:
                    driver.execute_script("window.scrollBy(0, 100);")
                    if post.is_displayed():
                        post.click()
                        break
                          

                # 없으면 오류제거하고 while True 반복 진행
                except NoSuchElementException:
                    pass


                # 스크롤 시작점=0, ?픽셀씩 내리기
                driver.execute_script("window.scrollBy(0, 430);")
                time.sleep(0.6)

                if time.time() - start_time_scroll > 60:
                    break  
            # while 코드 끝  
            # while 코드에서 클릭한 공고의 data crawling 코드
            try:
                # 공고 클릭 후 Data Crawling
                corp = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located(
                        (
                        By.XPATH, '//*[@id="__next"]/main/div[1]/div/section/header/div/div[1]/a'
                        )
                        )
                        ).text
                time.sleep(0.2)
                #더보기 클릭
                driver.find_element(
                    By.XPATH, '//*[@id="__next"]/main/div[1]/div/section/section/article[1]/div/button'
                    ).click()
                time.sleep(0.2)
                tec = driver.find_element(
                    By.XPATH, '//*[@id="__next"]/main/div[1]/div/section/section/article[1]/div/div[3]/p/span'
                    ).text.replace('\n',', ')
                time.sleep(0.2)
                car = driver.find_element(
                    By.XPATH, '//*[@id="__next"]/main/div[1]/div/section/header/div/div[1]/span[4]'
                    ).text
                time.sleep(0.2)
                post_url = [q.get_attribute('href') for q in driver.find_elements(By.TAG_NAME, 'link')][0]
                time.sleep(0.2)
                main = re.sub(
                    '\s', '', driver.find_element(
                        By.XPATH, '//*[@id="__next"]/main/div[1]/div/section/section/article[1]/div/div[2]/p/span'
                        ).text
                        ).replace('-', ', ')[1:]
                time.sleep(0.2)
                try:
                    address = driver.find_element(
                        By.XPATH, '//*[@id="__next"]/main/div[1]/div/section/section/article[4]/div/div/span'
                        ).text
                except:
                    address = driver.find_element(
                        By.XPATH, '//*[@id="__next"]/main/div[1]/div/section/section/article[5]/div/div/span'
                        ).text
                
            except:
                None
            
            # Data 인덱스 아래부터 concat
            wanted_temp = pd.DataFrame(
                [corp, address, button, main, car, tec, post_url],
                index = ['기업명', '주소', '직무', '자격요건', '직급', '이용하는기술스택/우대사항', '해당 페이지 URL']
                ).T
            wanted_res = pd.concat(
                [wanted_res, wanted_temp]
                )
            
            time.sleep(0.3)
            # 뒤로가기
            driver.back()
            time.sleep(0.3)

    finally:
        driver.quit()
    
    return wanted_res


In [ ]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time



# 실행 코드

final_wan_res = pd.DataFrame(columns=['기업명', '주소', '직무', '자격요건', '직급', '이용하는기술스택/우대사항', '해당 페이지 URL'])
for button in range(36, 39):
    # for start_post in range(처음 post, 최대 post, 로그인 페이지방지: 3 추천)
    for start_post in range(1, 300, 3):
        try:
            # end_post = start_post + 로그인 페이지 방지 숫자
            end_post = start_post + 3
            final_wan_res = pd.concat(
                [final_wan_res, wanted_crawling_program(start_post, end_post)]
                )
        
        except UnboundLocalError:
            break 


# 직무 숫자를 직무로 변경하는 코드

# 크롬옵션 설정
chrome_options = Options()
# 꺼짐 방지 (gqu가속 x)
chrome_options.add_argument('--disable-gpu') 
# 창 크기에 따라 XPATH가 달라져 크기 고정
chrome_options.add_argument("--start-maximized")
# 알림표시 제거
chrome_options.add_argument("--disable-notifications")
# 옵션 적용
driver = webdriver.Chrome(options=chrome_options)

url = "https://www.wanted.co.kr/wdlist/518?country=kr&job_sort=job.recommend_order&years=-1&locations=all"
# chrome url 넣어서 실행
driver.get(url)

# 직무 선택
search_job = WebDriverWait(driver, 10).until(
EC.presence_of_element_located(
    (
        By.XPATH, '//*[@id="__next"]/div[3]/article/div/div[2]/button'
        )
        )
        )
search_job.click()

# 직무 숫자에 맞춰 직무 list 만들기
job_description_list = []
for mung in range(2,39):
    job_description_list.append(driver.find_element(By.XPATH, f'//*[@id="__next"]/div[3]/article/div/div[2]/section/div[1]/div/button[{mung}]').text)

# 직무 list를 직무에 적용
for job_description in range(2,39):
    final_wan_res.loc[final_wan_res.직무 == job_description, '직무'] = job_description_list[job_description - 2]
    time.sleep(0.5)